In [ ]:
using Pkg
Pkg.add("Plots")
Pkg.add("Languages")
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("Query")
Pkg.add("LightGraphs")
# conda install -c conda-forge flann 
# Pkg.add("SGtSNEpi")
Pkg.add("StatsBase")
Pkg.add("Compose")
Pkg.add("GraphPlot")

# Lets see if we can identify duplicate or related projects

I.e. "Alberta" and "Government of Alberta" are the same thing.

In [ ]:
using Languages
using DataFrames
using CSV
using Query
using SparseArrays

df = CSV.read("../SAMPLE-ESTMA-data.csv", DataFrame)

eng_articles = articles(Languages.English())
eng_stops = stopwords(Languages.English())
simple_filter(x) = !(x in eng_stops || x in eng_articles)

In [ ]:
ENV["COLUMNS"] = 400
first(df, 5)

In [ ]:
payees_or_projects = unique(df.payee_project_name)

In [ ]:
# Vectorize and remove articles
vectorized_names = payees_or_projects .|> 
        lowercase .|> 
        split .|> 
        (x -> filter(simple_filter, x))

In [ ]:
aliases_df = DataFrame(orig = payees_or_projects, vectorized = vectorized_names)

In [ ]:
length(aliases_df.vectorized) - length(unique(aliases_df.vectorized))

### Note: already identified 200 likely duplicates here after removing articles and lowercasing

In [ ]:
first_aliases = aliases_df |>
    @groupby(_.vectorized) |>
    @filter(x -> length(x) > 1) |>
    @map(x -> [y.orig for y in x]) |>
    collect


In [ ]:
length(collect(Iterators.flatten(first_aliases)))

# Let's see if we can identify more duplicates

In [ ]:
# Count the common number of words
counts(a, b) = length(intersect(a,b))

vnames = unique(aliases_df.vectorized)

In [ ]:
@time begin

width = length(vnames)

m = zeros(Int64, (width, width))

# Symmetric matrix and skip the diagonal
# Still 32000000 entries!!!
for i in 1:width
    for j in (i+1):width
        val = counts(vnames[i], vnames[j])
        val == 0 && continue # skip array allocation if no overlap
        m[i,j] = m[j,i] = val
    end
end
# [counts(a,b) for a in vectorized_names, b in vectorized_names]

end

In [ ]:
m

In [ ]:
# Are there any with no neighbors?
count = 0
for i in 1:width
    if max(m[:,1]...) == 0
        count += 1
    end
end

In [ ]:
count

## Very high connectivity!

All the rows are connected to at least one neighbour. Let's see how many neighbours

In [ ]:
adjacent(i; v=vnames, m=m) = v[i] => getindex(v, findall(m[:,i] .> 0))

In [ ]:
last(adjacent(1))

In [ ]:
first(adjacent(1))

### Ahah! It's a few words (in this case "mining") which are super prevalent.

Let's plot a histogram of the terms, and remove some big ones to make this graph sparser. Like an ad-hoc [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)

In [ ]:
using Plots
using StatsBase

In [ ]:
all_words = collect(Iterators.flatten(vectorized_names))

In [ ]:
plot(countmap(all_words))

In [ ]:
sorted_counts = sort(collect(countmap(all_words)), by=last)

our_words = sorted_counts .|> first
our_counts = sorted_counts .|> last

plot(our_words, our_counts)

### A few very common words

This makes sense! Lets see what they are

In [ ]:
word_counts = countmap(all_words)

In [ ]:
filter(x -> last(x) > 100, word_counts)

### Great! We can remove some of these.

Some have semantic meaning (e.g. Alberta), but others, `-` or `municipal`, for instance, do not really contribute too much. We will filter them out and we should be left with a much sparser graph.

In [ ]:
removable = filter(x -> last(x) > 100, word_counts) |> keys |> collect

# Only alberta seems important in this list
deleteat!(removable, removable .== "alberta")

In [ ]:
filtered_vectorized_names = vectorized_names .|> 
    (y -> filter(x -> !(x in removable), y))

aliases_df.filtered = filtered_vectorized_names

In [ ]:
first(aliases_df, 5)

### As we did before, lets identify if any duplicated have appeared

In [ ]:
new_duplicates = let 
    old_duplicates = length(aliases_df.vectorized) - length(unique(aliases_df.vectorized))
    cur_duplicates = length(aliases_df.filtered) - length(unique(aliases_df.filtered))
    cur_duplicates - old_duplicates
end

#### 603 duplicates!!

In [ ]:
second_aliases = aliases_df |>
    @groupby(_.filtered) |>
    @filter(x -> length(x) > 1) |>
    @map(x -> [y.orig for y in x]) |>
    collect

## Let's keep going

In [ ]:
fnames = unique(aliases_df.filtered)

In [ ]:
@time begin

width = length(fnames)

m = zeros(Int64, (width, width))

# Symmetric matrix and skip the diagonal
# Still 32000000 entries!!!
for i in 1:width
    for j in (i+1):width
        val = counts(fnames[i], fnames[j])
        val == 0 && continue # skip array allocation if no overlap
        m[i,j] = m[j,i] = val
    end
end
# [counts(a,b) for a in vectorized_names, b in vectorized_names]

end

## Next lets create some adjacency matrices

You get a 1000x speedup by using SparseArrays

In [ ]:
sm = sparse(m)

# Initialize the vector
first_ten = repeat([sm], 10)
for i in 2:10
    first_ten[i] = first_ten[i-1] * sm
end

In [ ]:
g = SimpleGraph(sm)

In [ ]:
function getsubgraph(i, steps; g=g, seq=first_ten)
    # non-zero indices in column
    indices = Vector{Int}()
    for s in 1:steps
        v = (seq[s][:,i]).nzind
        indices = vcat(indices, v)
    end
    indices = unique(indices)
    
    return induced_subgraph(g, indices)
end 

In [ ]:
subg = getsubgraph(1, 3)

In [ ]:
gplot(first(subg))

In [ ]:
df2 = DataFrame(orig = payees_or_projects, vect = vectorized_names)

In [ ]:
using SparseArrays

In [ ]:
using LinearAlgebra

In [ ]:
l = [1,2,3]

In [ ]:
push!(l,4)

In [ ]:
append!(l,5)

In [ ]:
last(l)

In [ ]:
using LightGraphs

In [ ]:
using Compose, GraphPlot